## Objective

    .h5파일은 좋지만, Pandas가 좀 더 편하다. 매번 .h5파일에서 읽어와서 pandas.DataFrame으로 바꾸는 건 곤욕이다. 이걸 좀더 편하게 해주는 코드를 만들어보자
    

.h5파일에서 pandas처럼 바로 불러오는 Wrapper Class를 만드는게 목표이다. 

In [1]:
import h5py
import pandas as pd
import numpy as np

일단 만들어진 코드가 어떻게 돌아가는지를 보면 다음과 같다

In [2]:
import sys
sys.path.append("../")
from data_utils import DataLoader

In [3]:
# 전처리된 .h5 파일을 불러옴
dl = DataLoader("../data/prep/train_string.h5")

카카오의 데이터는 매우 크다. 그래서 메모리에 모두 올리는 건 매우 부담스럽다. 

그렇다면, hdf5에서 필요한 부분만 호출해서 올리는게 어떨까? 원래 hdf5의 설계 자체가 DB처럼
빠르게 파일에서 필요한 부분만 호출할 수 있도록 만들어졌다. 그래서 램보다는 느리지만, 그래도 제법 빠른 속도로 파일을 입출력 할 수 있다.

그런데, hdf5파일을 다루기 위해서는 h5py로 코드를 처리해야 한다. 제법 API가 잘 짜여져 있지만, 그래도 코드가 마냥 Simple하지만은 않다. 그래서 마치 DataFrame처럼 호출해서 가져올 수 있는 클래스를 만들고자 한다. 

### 사용법은 아래와 같다

사용법은 pandas의 index방식을 모방해서 만들었는데, 살짝 다르게 설계되었다.

#### 1) Call by Column

In [4]:
dl['pid']

0          Q4081781803
1          W4203425504
2          G4453903364
3          U4418629259
4          I4066071748
5          J4586931195
6          F4662379886
7          O3764058858
8          J3959473240
9          K4487826783
10         W4100190891
11         Y4608810761
12         G3382184545
13         J4364582444
14         I3502158308
15         R4216504551
16         M4659179049
17         T4480192202
18         Y4722777949
19         Y4583542454
20         Q4164621977
21         V4740525745
22         H2995437169
23         M4207737654
24         F4702878597
25         Y3242501673
26         Z2534298325
27         W4721558996
28         X3796883287
29         L4720929458
              ...     
8134788    I4571489417
8134789    L4462167687
8134790    G4465718172
8134791    Y3994693665
8134792    M4103937602
8134793    O4565369299
8134794    O2705040564
8134795    K3109845258
8134796    N4536410231
8134797    S4584755403
8134798    R3146172936
8134799    O3322467671
8134800    

#### 2) Call by Column and index

In [5]:
dl['pid',10]

0    W4100190891
Name: pid, dtype: object

#### 3) call by Column and slice

In [6]:
dl["pid",3:7]

0    U4418629259
1    I4066071748
2    J4586931195
3    F4662379886
Name: pid, dtype: object

#### 4) call string by Column

hdf5에서 가져올 때 기본적으로 byte 형태로 출력된다. 이를 dataloader에서는 받아서, utf-8로 디코딩을 해서 출력해주게 된다. 

In [7]:
dl["product",:10]

0    마우스 Wireless Mouse 2.4GHz game Ergonomic Desig...
1    [SB몰]스페리 Top-Sider Songfish Python 여성 Flats & ...
2         (1553)CRG-309 재생토너 캐논 LBP-5250K/5350K/6525K/
3                     중성펜/젤러펜 필기구 볼펜류 볼펜심제브라 Refill SK
4                   마이비 마이비  무형광 아기세탁망 원형 M (45cm)/빨래망
5                      031-893-8020 평택안중백병원장례식장 화환추천BZ
6                              스카치 초강력 폼양면 테이프 (대) 다용도
7    Continental CrossContact LX 스포츠 275/45R21 110W...
8                  [롯데백화점][에트로]토트 겸 숄더백 1H558-8259-150
9       [KB 5% 청구할인]LG 옵티머스G3 스크린 House 젤리 핸드폰케이스 F490
Name: product, dtype: object

#### 5) call by multiple column

In [8]:
dl[['pid','product'],:10]

,pid,product
0,Q4081781803,마우스 Wireless Mouse 2.4GHz game Ergonomic Desig...
1,W4203425504,[SB몰]스페리 Top-Sider Songfish Python 여성 Flats & ...
2,G4453903364,(1553)CRG-309 재생토너 캐논 LBP-5250K/5350K/6525K/
3,U4418629259,중성펜/젤러펜 필기구 볼펜류 볼펜심제브라 Refill SK
4,I4066071748,마이비 마이비 무형광 아기세탁망 원형 M (45cm)/빨래망
5,J4586931195,031-893-8020 평택안중백병원장례식장 화환추천BZ
6,F4662379886,스카치 초강력 폼양면 테이프 (대) 다용도
7,O3764058858,Continental CrossContact LX 스포츠 275/45R21 110W...
8,J3959473240,[롯데백화점][에트로]토트 겸 숄더백 1H558-8259-150
9,K4487826783,[KB 5% 청구할인]LG 옵티머스G3 스크린 House 젤리 핸드폰케이스 F490


#### 6) call All

In [9]:
# 이렇게 하면 되는데, 메모리 많이 잡아먹는다... 나는 이런 식으로 call 하진 않을란다
# dl[:,:]

#### 7) call by column and list

Random Order로도 Call 할 수 있도록 만들었다. (여기에 은근 시간이 많이 들었다...)

In [10]:
dl["product",[0,10,7,120]]

0    마우스 Wireless Mouse 2.4GHz game Ergonomic Desig...
1                   NT)디자인컷터날(BS-28P_점선)-갑(10개입) 사무용커터
2    Continental CrossContact LX 스포츠 275/45R21 110W...
3                     [GM데코] iy431-세계나라들의대표빵_브런치_노프레임_
Name: product, dtype: object